# FSRS4Anki v2.1.1 Optimizer

[![open in colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/open-spaced-repetition/fsrs4anki/blob/v2.1.1/fsrs4anki_optimizer.ipynb)

↑ Click the above button to open the optimizer on Google Colab.

> If you can't see the button and are located in the Chinese Mainland, please use a proxy or VPN.

Upload your **Anki Deck Package (.apkg)** file or **Anki Collection Package (.colpkg)** file on the `Left sidebar -> Files`, drag and drop your file in the current directory (not the `sample_data` directory). 

No need to include media. Need to include scheduling information. 

> If you use the latest version of Anki, please check the box `Support older Anki versions (slower/larger files)` when you export.

You can export it via `File -> Export...` or `Ctrl + E` in the main window of Anki.

Then replace the `filename` with yours in the next code cell. And set the `timezone` and `next_day_starts_at` which can be found in your preferences of Anki.

After that, just run all (`Runtime -> Run all` or `Ctrl + F9`) and wait for minutes. You can see the optimal parameters in section **3 Result**. Copy them, replace the parameters in `fsrs4anki_scheduler.js`, and paste them into the custom scheduling of your deck options (require Anki version >= 2.1.55).

**NOTE**: The default output is generated from my review logs. If you find the output is the same as mine, maybe your notebook hasn't run there.

In [1]:
# Here are some settings that you need to replace before running this optimizer.

filename = "ALL__Learning.apkg"
# If you upload deck file, replace it with your deck filename. E.g., ALL__Learning.apkg
# If you upload collection file, replace it with your colpgk filename. E.g., collection-2022-09-18@13-21-58.colpkg

# Replace it with your timezone. I'm in China, so I use Asia/Shanghai.
timezone = 'Asia/Shanghai'

# Replace it with your Anki's setting in Prefernces -> Scheduling.
next_day_starts_at = 4

# Replace it if you don't want the optimizer to use the review logs before a specific date.
revlog_start_date = "2006-10-05"


## 1 Build dataset

### 1.1 Extract Anki collection & deck file

In [2]:
import zipfile
# Extract the collection file or deck file to get the .anki21 database.
with zipfile.ZipFile(f'./{filename}', 'r') as zip_ref:
    zip_ref.extractall('./')
    print("Extract successfully!")


Extract successfully!


### 1.2 Create time-series feature

The following code cell will extract the review logs from your Anki collection and preprocess them to a trainset which is saved in `revlog_history.tsv`.

 The time-series features are important in optimizing the model's parameters. For more detail, please see my paper: https://www.maimemo.com/paper/

In [3]:
import sqlite3
import time
import tqdm
import pandas as pd
import os
from datetime import timedelta, datetime
from tqdm import tqdm

if os.path.isfile("collection.anki21b"):
    os.remove("collection.anki21b")
    raise Exception(
        "Please export the file with `support older Anki versions` if you use the latest version of Anki.")
elif os.path.isfile("collection.anki21"):
    con = sqlite3.connect("collection.anki21")
elif os.path.isfile("collection.anki2"):
    con = sqlite3.connect("collection.anki2")
else:
    raise Exception("Collection not exist!")
cur = con.cursor()
res = cur.execute("SELECT * FROM revlog")
revlog = res.fetchall()

df = pd.DataFrame(revlog)
df.columns = ['id', 'cid', 'usn', 'r', 'ivl',
              'last_lvl', 'factor', 'time', 'type']
df = df[(df['cid'] <= time.time() * 1000) &
        (df['id'] <= time.time() * 1000) &
        (df['id'] >= time.mktime(datetime.strptime(revlog_start_date, "%Y-%m-%d").timetuple()) * 1000)].copy()
df['create_date'] = pd.to_datetime(df['cid'] // 1000, unit='s')
df['create_date'] = df['create_date'].dt.tz_localize(
    'UTC').dt.tz_convert(timezone)
df['review_date'] = pd.to_datetime(df['id'] // 1000, unit='s')
df['review_date'] = df['review_date'].dt.tz_localize(
    'UTC').dt.tz_convert(timezone)
df.drop(df[df['review_date'].dt.year < 2006].index, inplace=True)
df.sort_values(by=['cid', 'id'], inplace=True, ignore_index=True)
df.to_csv("revlog.csv", index=False)
print("revlog.csv saved!")
df = df[(df['type'] == 0) | (df['type'] == 1)].copy()
df['real_days'] = df['review_date'] - timedelta(hours=next_day_starts_at)
df['real_days'] = pd.DatetimeIndex(df['real_days'].dt.floor('D')).to_julian_date()
df.drop_duplicates(['cid', 'real_days'], keep='first', inplace=True)
df['delta_t'] = df.real_days.diff()
df.dropna(inplace=True)
df['delta_t'] = df['delta_t'].astype(dtype=int)
df['i'] = 1
df['r_history'] = ""
df['t_history'] = ""
col_idx = {key: i for i, key in enumerate(df.columns)}


# code from https://github.com/L-M-Sherlock/anki_revlog_analysis/blob/main/revlog_analysis.py
def get_feature(x):
    for idx, log in enumerate(x.itertuples()):
        if idx == 0:
            x.iloc[idx, col_idx['delta_t']] = 0
        if idx == x.shape[0] - 1:
            break
        x.iloc[idx + 1, col_idx['i']] = x.iloc[idx, col_idx['i']] + 1
        x.iloc[idx + 1, col_idx['t_history']] = f"{x.iloc[idx, col_idx['t_history']]},{x.iloc[idx, col_idx['delta_t']]}"
        x.iloc[idx + 1, col_idx['r_history']] = f"{x.iloc[idx, col_idx['r_history']]},{x.iloc[idx, col_idx['r']]}"
    return x


tqdm.pandas()
df = df.groupby('cid', as_index=False).progress_apply(get_feature)
df["t_history"] = df["t_history"].map(lambda x: x[1:] if len(x) > 1 else x)
df["r_history"] = df["r_history"].map(lambda x: x[1:] if len(x) > 1 else x)
df.to_csv('revlog_history.tsv', sep="\t", index=False)
print("Trainset saved!")


revlog.csv saved!


100%|██████████| 5166/5166 [00:17<00:00, 299.33it/s]


Trainset saved!


## 2 Optimize parameter

### 2.1 Define the model

FSRS is a time-series model for predicting memory states.

In [4]:
import math
import sys
import torch
import numpy as np
from torch import nn
from sklearn.utils import shuffle

initStability = 1
initStabilityRatingFactor = 1
initDifficulty = 1
initDifficultyRatingFactor = -1
updateDifficultyRatingFactor = -1
difficultyMeanReversionFactor = 0.2
recallFactor = 3
recallDifficultyDecay = -0.8
recallStabilityDecay = -0.2
recallRetrievabilityFactor = 1.3
forgetFactor = 2.2
forgetDifficultyDecay = -0.3
forgetStabilityDecay = 0.3
forgetRetrievabilityFactor = 1.2

class FSRS(nn.Module):
    def __init__(self):
        super(FSRS, self).__init__()
        self.f_s = nn.Parameter(torch.FloatTensor([initStability, initStabilityRatingFactor]))
        # init stability
        self.f_d = nn.Parameter(torch.FloatTensor([initDifficulty, initDifficultyRatingFactor, updateDifficultyRatingFactor, difficultyMeanReversionFactor]))
        # init difficulty
        self.s_w = nn.Parameter(torch.FloatTensor([recallFactor, recallDifficultyDecay, recallStabilityDecay, recallRetrievabilityFactor, forgetFactor, forgetDifficultyDecay, forgetStabilityDecay, forgetRetrievabilityFactor]))
        self.zero = torch.FloatTensor([0.0])

    def forward(self, x, s, d):
        '''
        :param x: [review interval, review response]
        :param s: stability
        :param d: difficulty
        :return:
        '''
        if torch.equal(s, self.zero):
            # first learn, init memory states
            new_d = self.f_d[0] * (self.f_d[1] * (x[1] - 4) + 1)
            new_s = self.f_s[0] * (self.f_s[1] * (x[1] - 1) + 1)
        else:
            r = torch.exp(np.log(0.9) * x[0] / s)
            new_d = d + self.f_d[2] * (x[1] - 3)
            new_d = self.mean_reversion(self.f_d[0] * (- self.f_d[1] + 1), new_d)
            new_d = self.constrain(new_d)
            # recall
            if x[1] > 1:
                new_s = s * (1 + torch.exp(self.s_w[0]) * 
                            torch.pow(new_d, self.s_w[1]) *
                            torch.pow(s, self.s_w[2]) *
                            (torch.exp((1 - r) * self.s_w[3]) - 1))
            # forget
            else:
                new_s = self.s_w[4] * torch.pow(new_d, self.s_w[5]) * torch.pow(s, self.s_w[6]) * torch.exp((1 - r) * self.s_w[7])
        return new_s, new_d

    def loss(self, s, t, r):
        return - (r * np.log(0.9) * t / s + (1 - r) * torch.log(1 - torch.exp(np.log(0.9) * t / s)))

    def constrain(self, d):
        return torch.relu(d - 1) + 1

    def mean_reversion(self, init, current):
        return self.f_d[3] * init + (1-self.f_d[3]) * current


class WeightClipper(object):
    def __init__(self, frequency=1):
        self.frequency = frequency

    def __call__(self, module):
        if hasattr(module, 'f_s'):
            w = module.f_s.data
            w[0] = w[0].clamp(0.1, 10)  # initStability
            w[1] = w[1].clamp(0.01, 10)  # initStabilityRatingFactor
            module.f_s.data = w
        if hasattr(module, 'f_d'):
            w = module.f_d.data
            w[0] = w[0].clamp(1, 10)  # initDifficulty
            w[1] = w[1].clamp(-10, -0.01)  # initDifficultyRatingFactor
            w[2] = w[2].clamp(-10, -0.01)  # updateDifficultyRatingFactor
            w[3] = w[3].clamp(0, 1)  # difficultyMeanReversionFactor
            module.f_d.data = w
        if hasattr(module, 's_w'):
            w = module.s_w.data
            w[0] = w[0].clamp(0, 5)  # recallFactor
            w[1] = w[1].clamp(-2, -0.01)  # recallDifficultyDecay
            w[2] = w[2].clamp(-2, -0.01)  # recallStabilityDecay
            w[3] = w[3].clamp(0.01, 2)  # recallRetrievabilityFactor
            w[4] = w[4].clamp(0, 5)  # forgetFactor
            w[5] = w[5].clamp(-2, -0.01)  # forgetDifficultyDecay
            w[6] = w[6].clamp(0.01, 1)  # forgetStabilityDecay
            w[7] = w[7].clamp(0.01, 2)  # forgetRetrievabilityFactor
            module.s_w.data = w


def lineToTensor(line):
    ivl = line[0].split(',')
    response = line[1].split(',')
    tensor = torch.zeros(len(response), 2)
    for li, response in enumerate(response):
        tensor[li][0] = int(ivl[li])
        tensor[li][1] = int(response)
    return tensor


### 2.2 Train the model

The `revlog_history.tsv` generated before will be used for training the FSRS model.

In [5]:
model = FSRS()
clipper = WeightClipper()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)

dataset = pd.read_csv("./revlog_history.tsv", sep='\t', index_col=None)
dataset = dataset[(dataset['i'] > 1) & (dataset['delta_t'] > 0) & (dataset['t_history'].str.count(',0') == 0)]
dataset['tensor'] = dataset.progress_apply(lambda x: lineToTensor(
    list(zip([x['t_history']], [x['r_history']]))[0]), axis=1)
print("Tensorized!")


pre_train_set = dataset[dataset['i'] == 2]
# pretrain
n_epoch = 5

for k in range(n_epoch):
    pre_train_set = shuffle(pre_train_set, random_state=2022 + k)
    epoch_len = len(pre_train_set)
    for i, (_, row) in enumerate(tqdm(pre_train_set.iterrows(), total=epoch_len, desc="train", colour="red")):
        model.train()
        optimizer.zero_grad()
        output_t = [(model.zero, model.zero)]
        for input_t in row['tensor']:
            output_t.append(model(input_t, *output_t[-1]))
        loss = model.loss(output_t[-1][0], row['delta_t'],
                            {1: 0, 2: 1, 3: 1, 4: 1}[row['r']])
        if np.isnan(loss.data.item()):
            # Exception Case
            print(row, output_t)
            raise Exception('error case')
        loss.backward()
        optimizer.step()
        model.apply(clipper)

for name, param in model.named_parameters():
    if name == "f_s":
        param.requires_grad = False
    print(f"{name}: {list(map(lambda x: round(float(x), 4),param))}")

n_epoch = 1
train_set = dataset[dataset['i'] > 2]
print_len = max(train_set.shape[0] // 10, 1)

for k in range(n_epoch):
    train_set = shuffle(train_set, random_state=2022 + k)
    epoch_len = len(train_set)
    for i, (_, row) in enumerate(tqdm(train_set.iterrows(), total=epoch_len, desc="train", colour="red")):
        model.train()
        optimizer.zero_grad()
        output_t = [(model.zero, model.zero)]
        for input_t in row['tensor']:
            output_t.append(model(input_t, *output_t[-1]))
        loss = model.loss(output_t[-1][0], row['delta_t'],
                          {1: 0, 2: 1, 3: 1, 4: 1}[row['r']])
        if np.isnan(loss.data.item()):
            # Exception Case
            print(row, output_t)
            raise Exception('error case')
        loss.backward()
        optimizer.step()
        model.apply(clipper)

        if (k * epoch_len + i) % print_len == 0:
            print(f"iteration: {k * epoch_len + i + 1}")
            for name, param in model.named_parameters():
                print(f"{name}: {list(map(lambda x: round(float(x), 4),param))}")


initStability, initStabilityRatingFactor = map(
    lambda x: round(float(x), 4), dict(model.named_parameters())['f_s'].data)
initDifficulty, initDifficultyRatingFactor, updateDifficultyRatingFactor, difficultyMeanReversionFactor = map(
    lambda x: round(float(x), 4), dict(model.named_parameters())['f_d'].data)
recallFactor, recallDifficultyDecay, recallStabilityDecay, recallRetrievabilityFactor, forgetFactor, forgetDifficultyDecay, forgetStabilityDecay, forgetRetrievabilityFactor = map(
    lambda x: round(float(x), 4), dict(model.named_parameters())['s_w'].data)

print("\nTraining finished!")


100%|██████████| 56910/56910 [00:03<00:00, 15362.11it/s]


Tensorized!


train:   0%|          | 26/56910 [00:00<03:40, 258.03it/s]

iteration: 1
f_s: [1.0005, 1.0005]
f_d: [1.0, -0.9995, -0.9995, 0.2005]
s_w: [3.0005, -0.7995, -0.1995, 1.3005, 2.2005, -0.2995, 0.3005, 1.2005]


train:  10%|█         | 5741/56910 [00:14<02:01, 421.39it/s]

...

iteration: 45529
f_s: [1.7054, 1.9824]
f_d: [1.0233, -0.9943, -1.2296, 0.0866]
s_w: [3.0414, -1.0547, -0.0734, 1.314, 2.2104, -0.2478, 0.3548, 1.1255]


train:  90%|█████████ | 51267/56910 [02:17<00:16, 338.00it/s]

iteration: 51220
f_s: [1.6519, 1.9887]
f_d: [1.006, -0.9738, -1.2296, 0.0881]
s_w: [3.062, -1.0546, -0.061, 1.3333, 2.2051, -0.2531, 0.3472, 1.1081]


train: 100%|██████████| 56910/56910 [02:35<00:00, 365.71it/s]


Training finished!


## 3 Result

Copy the optimal parameters for FSRS for you in the output of next code cell after running.

In [6]:
print(f"var f_s = [{initStability},{initStabilityRatingFactor}];")
print(f"var f_d = [{initDifficulty},{initDifficultyRatingFactor},{updateDifficultyRatingFactor},{difficultyMeanReversionFactor}];")
print(f"var s_w = [{recallFactor},{recallDifficultyDecay},{recallStabilityDecay},{recallRetrievabilityFactor},{forgetFactor},{forgetDifficultyDecay},{forgetStabilityDecay},{forgetRetrievabilityFactor}];")

var f_s = [1.6543,1.9849];
var f_d = [1.006,-0.9394,-1.2311,0.0923];
var s_w = [3.0788,-1.0484,-0.0571,1.3462,2.2089,-0.2442,0.3476,1.099];


You can see the memory states and intervals generated by FSRS as if you press the good in each review at the due date scheduled by FSRS.

In [7]:
requestRetention = 0.9  # recommended setting: 0.8 ~ 0.9


class Collection:
    def __init__(self):
        self.model = model

    def states(self, t_history, r_history):
        with torch.no_grad():
            line_tensor = lineToTensor(list(zip([t_history], [r_history]))[0])
            output_t = [(self.model.zero, self.model.zero)]
            for input_t in line_tensor:
                output_t.append(self.model(input_t, *output_t[-1]))
            return output_t[-1]


my_collection = Collection()
print("1:again, 2:hard, 3:good, 4:easy\n")
for first_rating in (1,2,3,4):
    print(f'first rating: {first_rating}')
    t_history = "0"
    d_history = "0"
    r_history = f"{first_rating}"  # the first rating of the new card
    # print("stability, difficulty, lapses")
    for i in range(15):
        states = my_collection.states(t_history, r_history)
        # print('{0:9.2f} {1:11.2f} {2:7.0f}'.format(
            # *list(map(lambda x: round(float(x), 4), states))))
        next_t = round(float(np.log(requestRetention)/np.log(0.9) * states[0]))
        difficulty = round(float(states[1]), 1)
        t_history += f',{int(next_t)}'
        d_history += f',{difficulty}'
        r_history += f",3"
    print(f"rating history: {r_history}")
    print(f"interval history: {t_history}")
    print(f"difficulty history: {d_history}")
    print('')


1:again, 2:hard, 3:good, 4:easy

first rating: 1
rating history: 1,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
interval history: 0,2,3,6,10,18,33,60,108,195,352,634,1138,2035,3622,6414
difficulty history: 0,3.8,3.7,3.5,3.4,3.2,3.1,3.0,2.9,2.8,2.7,2.7,2.6,2.5,2.5,2.4

first rating: 2
rating history: 2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
interval history: 0,5,10,19,37,72,139,266,504,949,1771,3275,6003,10900,19606,34928
difficulty history: 0,2.9,2.8,2.7,2.7,2.6,2.5,2.5,2.4,2.4,2.3,2.3,2.3,2.2,2.2,2.2

first rating: 3
rating history: 3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
interval history: 0,8,19,44,99,218,467,978,2004,4022,7914,15282,28985,54043,99131,179022
difficulty history: 0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0

first rating: 4
rating history: 4,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
interval history: 0,12,41,129,373,1001,2521,6010,13650,29702,62211,125918,247134,471701,877783,1596050
difficulty history: 0,1.0,1.1,1.2,1.2,1.3,1.4,1.4,1.5,1.5,1.6,1.6,1.6,1.7,1.7,1.7



You can change the `test_rating_sequence` to see the scheduling intervals in different ratings.

In [8]:
test_rating_sequence = "3,1,1,3,3,3,3,1,3,3,3,3,3,3,3,3,1"
requestRetention = 0.9  # recommended setting: 0.8 ~ 0.9
easyBonus = 1.3
hardInterval = 1.2

t_history = "0"
d_history = "0"
for i in range(len(test_rating_sequence.split(','))):
    rating = test_rating_sequence[2*i]
    last_t = int(t_history.split(',')[-1])
    r_history = test_rating_sequence[:2*i+1]
    states = my_collection.states(t_history, r_history)
    print(states)
    next_t = max(1,round(float(np.log(requestRetention)/np.log(0.9) * states[0])))
    if rating == '4':
        next_t = round(next_t * easyBonus)
    elif rating == '2':
        next_t = round(last_t * hardInterval)
    t_history += f',{int(next_t)}'
    difficulty = round(float(np.log(requestRetention)/np.log(0.9) * states[1]), 1)
    d_history += f',{difficulty}'
print(f"rating history: {test_rating_sequence}")
print(f"interval history: {t_history}")
print(f"difficulty history: {d_history}")

(tensor(8.2213), tensor(1.9510))
(tensor(3.6043), tensor(4.1858))
(tensor(2.4923), tensor(6.2143))
(tensor(3.4277), tensor(5.8206))
(tensor(4.9015), tensor(5.4632))
(tensor(7.4729), tensor(5.1389))
(tensor(11.2073), tensor(4.8445))
(tensor(3.5671), tensor(6.8121))
(tensor(5.2437), tensor(6.3632))
(tensor(7.4362), tensor(5.9558))
(tensor(10.6537), tensor(5.5859))
(tensor(15.9462), tensor(5.2503))
(tensor(23.9526), tensor(4.9456))
(tensor(36.4160), tensor(4.6690))
(tensor(55.7545), tensor(4.4180))
(tensor(86.7965), tensor(4.1902))
(tensor(7.4459), tensor(6.2183))
rating history: 3,1,1,3,3,3,3,1,3,3,3,3,3,3,3,3,1
interval history: 0,8,4,2,3,5,7,11,4,5,7,11,16,24,36,56,87,7
difficulty history: 0,2.0,4.2,6.2,5.8,5.5,5.1,4.8,6.8,6.4,6.0,5.6,5.3,4.9,4.7,4.4,4.2,6.2
